In [89]:
import pandas as pd
import os
import plotly.express as px
from Functions.date_converter import convert_to_datetime
from Functions.data_processing import get_clean_df
from Functions.data_processing import AssetManager
from Functions.locations import Locations

In [90]:
# get file names
file_names = os.listdir('data')

# load excel (.xls) files to list
dfs = [pd.read_excel('data/' + file_name) for file_name in file_names if file_name.endswith('.xls')]

# for df in dfs: remove the first 4 rows and make row 4 the header
for df in dfs:
    df.columns = df.iloc[3]
    df.drop(df.index[:4], inplace=True)

# concatenate the dataframes into one
transactions = pd.concat(dfs, ignore_index=True)
transactions.drop_duplicates(ignore_index=True, inplace=True)
transactions['Date'] = transactions['Action Date'].apply(lambda x: convert_to_datetime(x).strftime('%d-%m-%Y'))
transactions_dict = {date: transactions[transactions['Date'] == date] for date in transactions['Date'].unique()}

In [91]:
def get_mw(transactions, location):
    update = AssetManager(4, location)
    update.Transactions_records = transactions
    update.update_tool_status()
    update.get_overdue('08-06-2023')
    return update.Overdue.shape[0]

In [92]:
#get graph data by simulating daily updates
def get_graph_data(transactions_dict, location):
    transactions_dict = {date: transactions_dict[date] for date in reversed(transactions_dict)}
    transactions_list = []
    graph_data = {}
    for date, transactions in transactions_dict.items():
        transactions_list.append(get_clean_df(transactions))
        graph_data[date] = get_mw(transactions_list, location)
    return pd.DataFrame.from_dict(graph_data, orient='index')

In [93]:
#plot graph data using plotly
for location in Locations:
    graph_data_df = get_graph_data(transactions_dict, location)
    fig = px.line(graph_data_df, x=graph_data_df.index, y=0)
    fig.update_layout(title_text=location)
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(title_text='Number of Tools overdue')
    fig.show()